In [ ]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
## Dataset

In [ ]:
all_clusters_df = pd.read_excel('/kaggle/input/allclusters523/all_clusters_df523.xlsx',header=0)

In [ ]:
import random
sentence_a = []
sentence_b = []
label = []

for i in range(len(all_clusters_df)):
    text = all_clusters_df.iloc[i]['Doc']
    BC = all_clusters_df.iloc[i]['Big_Cluster']
    SC = all_clusters_df.iloc[i]['Small_Cluster']
    sentences = [
        sentence for sentence in text.split('.') if sentence != ''
    ]
    num_sentences = len(sentences)
    if BC != -1 and SC != -1:
        for j in range(num_sentences):
            for k in range(num_sentences):
                if j != k:
                    sentence_a.append(sentences[k])
                    sentence_b.append(sentences[j])

In [ ]:
pd.set_option('display.max_colwidth', None) # Run this if you want to display the whole content
# pd.set_option('display.max_colwidth', 50) # Run this to return to default display options

In [ ]:
import random
num_rows = 15000
sentence_a_selected = []
sentence_b_selected = []

for _ in range(num_rows):
    random_index = random.randint(0, len(sentence_a)-1)
    sentence_a_selected.append(sentence_a[random_index])
    sentence_b_selected.append(sentence_b[random_index])

In [ ]:
## BART

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base", timeout=30)

In [ ]:
inputs_a = tokenizer(sentence_a_selected, truncation=True, padding=True, return_tensors='pt')
inputs_b = tokenizer(sentence_b_selected, truncation=True, padding=True, return_tensors='pt')

In [ ]:
import torch

class SentencePairDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.input_ids)

dataset = SentencePairDataset(inputs_a['input_ids'], inputs_a['attention_mask'], inputs_b['input_ids'])

In [ ]:
batch_size = 16
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [ ]:
val_data  = """Doctors should be able to edit their public information.
Doctors should be able to create patient accounts and delete those accounts.
Doctors should be able to add, modify, and delete patient medical notes.
Doctors should be able to add and delete patient's measured vital parameters.
Doctors should be able to define and edit mandatory parameters for patients.
Doctors should be able to define and edit parameter ranges for patients.
Doctors should be able to send alarms to the first aid team with patient's name, location, alarm level, and diagnosis.
Doctors should be able to access patient's medical notes, vital parameters, and images or files (on the web).
First aid should be able to delete alarms when they are acted upon in the web application.
The mobile application should display patient's medical notes (without images or other attachment files) and all measurements of patient's vital parameters to the doctor in charge.
The mobile application should allow the patient to input a new measurement of their vital parameters and enforce the input of mandatory vital parameters (if defined).
The web application should display patient's medical notes (with images or other attachment files) and all measurements of vital parameters to the doctor in charge of the patient.
The web application should allow the patient to input a new measurement of their vital parameters and enforce the input of mandatory vital parameters (if defined).
Doctors should be able to define and edit mandatory parameters for patients.
Doctors should be able to define and edit parameter ranges for patients.
Doctors should be able to send alarms to the first aid team with patient's name, location, alarm level, and diagnosis.
"""
sentences = [
        sentence for sentence in val_data.split('.') if sentence != ''
        ]

for sentence in sentences:
    encoded_inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors='pt').to(device)

    # Generate predictions
    outputs = model.generate(input_ids=encoded_inputs.input_ids, attention_mask=encoded_inputs.attention_mask)

    # Decode the predicted tensor
    predicted_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the predicted sentence
    print( predicted_sentence)

In [ ]:
E1 = [0.58, 0.80, 0.92, 0.33, 0.77, 0.42, 0.66, 0.48, 0.76]
E2 = [0.73, 0.50, 0.75, 0.66, 0.53, 0.5, 0.75, 0.43, 0.59]
E3 = [0.80, 0.75, 0.58, 0.67, 0.54, 0.80, 0.75, 0.66, 0.35]
E4 = [0.55, 0.45, 0.72, 0.44, 0.33, 0.62, 0.47, 0.37, 0.42]
E5 = [0.38, 0.15, 0.46, 0.23, 0.25, 0.36, 0.53, 0.52, 0.54]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
projects = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9']
# values = [0.58, 0.73, 0.92, 0.5, 0.75, 0.42, 0.66, 0.43,0.76,]
values = E3

# Calculate average value
average_value = np.mean(values)

# Create the bar plot
fig, ax = plt.subplots()
ax.bar(projects, values)

# Add average value line
ax.axhline(average_value, color='red', linestyle='--', label='Average Relevance')

# Set labels and title
ax.set_xlabel('Projects')
ax.set_ylabel('Relevance')
ax.set_title('Relevance per project after 3 epochs')

# Add legend
ax.legend()

# Display the plot
plt.show()

In [ ]:
averages = [0.63, 0.6, 0.63, 0.48, 0.38]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

epochs = [1, 2, 3, 4, 5]

# Calculate average value
average_value = np.mean(averages)

# Create the bar plot
fig, ax = plt.subplots()
ax.bar(epochs, averages)

# Add average value line
ax.axhline(average_value, color='red', linestyle='--', label='Average Relevance')

# Set labels and title
ax.set_xlabel('Epochs')
ax.set_ylabel('Average Relevance')
ax.set_title('Average Relevance per number of epochs')

# Add legend
ax.legend()

# Display the plot
plt.show()